In [1]:
################################################################
## READ ME                                                    ##
##                                                            ##
## Instruções                                                 ##
##                                                            ##
## WASD: Movimentam o personagem                              ##
## Mouse: Controla a Câmera                                   ##
## Space: Eleva o personagem                                  ##
## L-Shift: Baixa o personagem                                ##
## 1-4: Liga e desliga luzes                                  ##
##                                                            ## 
##                                                            ##
## Divirta-se!                                                ##
##                                                            ##
##                                                            ##
## Gabriel D. F. Mello                                        ##
##                                                            ##
################################################################

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [3]:
class Config:
    def __init__(self):
        self.v = []
        self.vn = []
        self.f = []
        self.fn = []
        self.num = 0

In [4]:
def export(path):
    config = Config()
    
    f = open(path, "r")
    for line in f:
        split = line.split(" ")
        
        # Vertice
        if(split[0] == "v"):
            config.v.append([float(split[1]), float(split[2]), float(split[3])])
            
        # Normal
        if(split[0] == "vn"):
            config.vn.append([float(split[1]), float(split[2]), float(split[3])])
            
        # Face
        if(split[0] == "f"):
            
            index = [0, 0, 0]
            
            index[0] = split[1].split("//")
            index[1] = split[2].split("//")
            index[2] = split[3].split("//")
            
            for i in range(0,3):
                for j in range (0,3):
                    config.f.append(config.v[int(index[i][0])-1][j])
                    config.num+=1
                    config.fn.append(config.vn[int(index[i][1])-1][j])
            
            
    return config

In [5]:
Window = None
Shader_programm = None
Shader_programm2 = None
VaoConfig = None
Vao = None
WIDTH = 800
HEIGHT = 600

In [6]:
Tempo_entre_frames = 0 

Cam_speed = 2.0 #velocidade da camera, 1 unidade por segundo
Cam_yaw_speed = 0.3
Cam_pitch_speed = 0.1
Cam_pos = np.array([0.0, 0.0, 3.0]) #posicao inicial da câmera
Cam_yaw = 0.0 #ângulo de rotação da câmera em x
Cam_pitch = 0.0 #ângulo de rotação da câmera em y

In [7]:
def redimensionaCallback(window, w, h):
    global WIDTH, HEIGHT
    WIDTH = w
    HEIGHT = h

In [8]:
def inicializaOpenGL():
    global Window, WIDTH, HEIGHT

    #Inicializa GLFW
    glfw.init()

    #Criação de uma janela
    Window = glfw.create_window(WIDTH, HEIGHT, "Trabalho Final", None, None)
    if not Window:
        glfw.terminate()
        exit()

    glfw.set_window_size_callback(Window, redimensionaCallback)
    glfw.make_context_current(Window)

    print("Placa de vídeo: ",OpenGL.GL.glGetString(OpenGL.GL.GL_RENDERER))
    print("Versão do OpenGL: ",OpenGL.GL.glGetString(OpenGL.GL.GL_VERSION))

In [9]:
def inicializaMesa():

    global Vao
    # Vao do cubo
    Vao = glGenVertexArrays(1)
    glBindVertexArray(Vao)

    global VaoConfig
    VaoConfig = export("Table Chairs CG.obj")
    # VBO dos vértices do quadrado
    points = VaoConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [10]:
def inicializaCaneca():
    global VaoMug
    # Vao do cubo
    VaoMug = glGenVertexArrays(1)
    glBindVertexArray(VaoMug)

    global VaoMugConfig
    VaoMugConfig = export("Mug CG.obj")
    # VBO dos vértices do quadrado
    points = VaoMugConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoMugConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [11]:
def inicializaPiso():
    global VaoPiso
    # Vao do cubo
    VaoPiso = glGenVertexArrays(1)
    glBindVertexArray(VaoPiso)

    global VaoPisoConfig
    VaoPisoConfig = export("Floor CG.obj")
    # VBO dos vértices do quadrado
    points = VaoPisoConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoPisoConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [12]:
def inicializaParede():

    global VaoWall
    # Vao do cubo
    VaoWall = glGenVertexArrays(1)
    glBindVertexArray(VaoWall)

    global VaoWallConfig
    VaoWallConfig = export("Wall CG.obj")
    # VBO dos vértices do quadrado
    points = VaoWallConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoWallConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [13]:
def inicializaSofa():

    global VaoSofa
    # Vao do cubo
    VaoSofa = glGenVertexArrays(1)
    glBindVertexArray(VaoSofa)

    global VaoSofaConfig
    VaoSofaConfig = export("Couch CG.obj")
    # VBO dos vértices do quadrado
    points = VaoSofaConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoSofaConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [14]:
def inicializaLampada():

    global VaoLampada
    # Vao do cubo
    VaoLampada = glGenVertexArrays(1)
    glBindVertexArray(VaoLampada)

    global VaoLampadaConfig
    VaoLampadaConfig = export("Light CG.obj")
    # VBO dos vértices do quadrado
    points = VaoLampadaConfig.f
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das normais
    normais = VaoLampadaConfig.fn
    normais = np.array(normais, dtype=np.float32)
    nvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, nvbo)
    glBufferData(GL_ARRAY_BUFFER, normais, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [15]:
def inicializaObjetos():
    inicializaMesa()
    inicializaCaneca()
    inicializaPiso()
    inicializaParede()
    inicializaLampada()
    inicializaSofa()

In [16]:
def inicializaShaders():
    global Shader_programm
    # Especificação do Vertex Shader:
    vertex_shader = """
        #version 400
        layout(location = 0) in vec3 vertex_posicao;
        layout(location = 1) in vec3 vertex_normal;
        out vec3 vertex_posicao_cam, vertex_normal_cam;
        uniform mat4 matriz, view, proj;
        void main () {
            vertex_posicao_cam = vec3 (view * matriz * vec4 (vertex_posicao, 1.0)); //posição do objeto em relação a CÂMERA
            vertex_normal_cam = vec3 (view *  matriz * vec4 (vertex_normal, 0.0)); //normais do objeto em relação a CÂMERA
            gl_Position = proj * view * matriz * vec4 (vertex_posicao, 1.0);
        }
    """
    vs = OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER)
    if not glGetShaderiv(vs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(vs, 512, None)
        print("Erro no vertex shader:\n", infoLog)

    # Especificação do Fragment Shader:
    fragment_shader = """
        #version 400
		in vec3 vertex_posicao_cam, vertex_normal_cam;
        
        //propriedades de uma luz pontual
        uniform vec3 luz_posicao;
        uniform vec3 Ls;// luz especular
		uniform vec3 Ld;// luz difusa
		uniform vec3 La;// luz ambiente

        //propriedades de reflexão da superficie
		uniform vec3 Ks;//reflexão especular
		uniform vec3 Kd;//reflexão difusa
		uniform vec3 Ka;//reflexão ambiente
        uniform float especular_exp;//expoente especular
        
        uniform mat4 view;
		out vec4 frag_colour;
		void main () {
       
            vec3 Ia = La * Ka;

            vec3 luz_posicao_cam = vec3 (view * vec4 (luz_posicao, 1.0));//posicao da luz em relação a câmera
            vec3 luz_vetor_cam = luz_posicao_cam - vertex_posicao_cam;//vetor apontando para a luz em relação a posicao do vértice
            
            vec3 luz_vetor_cam_normalizada = normalize(luz_vetor_cam);//vetor da luz normalizada
            
            vec3 vertex_normal_cam_normalizada = normalize(vertex_normal_cam);
            
            float cosseno_difusa = dot(vertex_normal_cam_normalizada,luz_vetor_cam_normalizada);//cosseno do angulo entre o vetor da luz e a normal da superficie
            
            vec3 Id = Ld * Kd * cosseno_difusa;


            
            vec3 luz_reflexao_vetor_cam = reflect(-luz_vetor_cam_normalizada, vertex_normal_cam_normalizada);
            
            
            vec3 superficie_camera_vetor = normalize(-vertex_posicao_cam);
            
            
            float cosseno_especular = dot(luz_reflexao_vetor_cam, superficie_camera_vetor);
            cosseno_especular = max(cosseno_especular, 0.0);//se o cosseno der negativo, atribui 0 para ele
            
            
            float fator_especular = pow (cosseno_especular, especular_exp);
            

            vec3 Is = Ls * Ks * fator_especular;
            
		    frag_colour = vec4 (Ia+Id+Is,1.0);
		}
    """
    fs = OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs, 512, None)
        print("Erro no fragment shader:\n", infoLog)

    # Especificação do Shader Programm:
    Shader_programm = OpenGL.GL.shaders.compileProgram(vs, fs)
    if not glGetProgramiv(Shader_programm, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_programm, 512, None)
        print("Erro na linkagem do shader:\n", infoLog)

    glDeleteShader(vs)
    glDeleteShader(fs)

In [17]:
def inicializaShaders2():
    global Shader_programm2
    # Especificação do Vertex Shader:
    vertex_shader = """
        #version 400
        layout(location = 0) in vec3 vertex_posicao;
        layout(location = 1) in vec3 vertex_normal;
        out vec3 vertex_posicao_cam, vertex_normal_cam;
        uniform mat4 matriz, view, proj;
        void main () {
            vertex_posicao_cam = vec3 (view * matriz * vec4 (vertex_posicao, 1.0)); //posição do objeto em relação a CÂMERA
            vertex_normal_cam = vec3 (view *  matriz * vec4 (vertex_normal, 0.0)); //normais do objeto em relação a CÂMERA
            gl_Position = proj * view * matriz * vec4 (vertex_posicao, 1.0);
        }
    """
    vs = OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER)
    if not glGetShaderiv(vs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(vs, 512, None)
        print("Erro no vertex shader:\n", infoLog)

    # Especificação do Fragment Shader:
    fragment_shader = """
        #version 400
		in vec3 vertex_posicao_cam, vertex_normal_cam;
        
        //propriedades de uma luz pontual
        uniform vec3 luz_posicao;
        uniform vec3 Ls;// luz especular
		uniform vec3 Ld;// luz difusa
		uniform vec3 La;// luz ambiente
        vec3 L1;

        //propriedades de reflexão da superficie
		uniform vec3 Ks;//reflexão especular
		uniform vec3 Kd;//reflexão difusa
		uniform vec3 Ka;//reflexão ambiente
        uniform float especular_exp;//expoente especular
        
        //propriedades de uma luz pontual
        uniform vec3 luz_posicao2;
        uniform vec3 Ls2;// luz especular
		uniform vec3 Ld2;// luz difusa
		uniform vec3 La2;// luz ambiente
        vec3 L2;
        
        uniform vec3 luz_posicao3;
        uniform vec3 Ls3;// luz especular
		uniform vec3 Ld3;// luz difusa
		uniform vec3 La3;// luz ambiente
        vec3 L3;

        
        uniform mat4 view;
        uniform int toggle = 0;
        
        
		out vec4 frag_colour;
        
        vec3 calc_luz(vec3 luz_posicao, vec3 La, vec3 Ld, vec3 Ls){
            vec3 Ia = La * Ka;

            vec3 luz_posicao_cam = vec3 (view * vec4 (luz_posicao, 1.0));//posicao da luz em relação a câmera
            vec3 luz_vetor_cam = luz_posicao_cam - vertex_posicao_cam;//vetor apontando para a luz em relação a posicao do vértice
            
            vec3 luz_vetor_cam_normalizada = normalize(luz_vetor_cam);//vetor da luz normalizada
            
            vec3 vertex_normal_cam_normalizada = normalize(vertex_normal_cam);
            
            float cosseno_difusa = dot(vertex_normal_cam_normalizada,luz_vetor_cam_normalizada);//cosseno do angulo entre o vetor da luz e a normal da superficie
            
            vec3 Id = Ld * Kd * cosseno_difusa;


            
            vec3 luz_reflexao_vetor_cam = reflect(-luz_vetor_cam_normalizada, vertex_normal_cam_normalizada);
            
            
            vec3 superficie_camera_vetor = normalize(-vertex_posicao_cam);
            
            
            float cosseno_especular = dot(luz_reflexao_vetor_cam, superficie_camera_vetor);
            cosseno_especular = max(cosseno_especular, 0.0);//se o cosseno der negativo, atribui 0 para ele
            
            
            float fator_especular = pow (cosseno_especular, especular_exp);
            

            vec3 Is = Ls * Ks * fator_especular;
            
            return (Ia + Id + Is);
        }
        
		void main () 
        { 
        // Luz 1
            if((toggle == 0)||(toggle == 1)){
                L1 = calc_luz(luz_posicao, La, Ld, Ls);
            }
            
            
        // Luz 2
            if((toggle == 0)||(toggle == 2)){
                L2 = calc_luz(luz_posicao2, La2, Ld2, Ls2);
            }
            
            
        // Luz 3
            if((toggle == 0)||(toggle == 3)){
                L3 = calc_luz(luz_posicao3, La3, Ld3, Ls3);
            }
            
        // Luz final
        
            if(toggle == 0){
                frag_colour = vec4 (L1 + L2 + L3,1.0);
            }
            else if (toggle == 1){
                frag_colour = vec4 (L1, 1.0);
            }
            else if (toggle == 2){
                frag_colour = vec4 (L2, 1.0);
            }
            else{
                frag_colour = vec4 (L3, 1.0);
            }
		}
    """
    fs = OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs, 512, None)
        print("Erro no fragment shader:\n", infoLog)

    # Especificação do Shader Programm:
    Shader_programm2 = OpenGL.GL.shaders.compileProgram(vs, fs)
    if not glGetProgramiv(Shader_programm2, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_programm2, 512, None)
        print("Erro na linkagem do shader:\n", infoLog)

    glDeleteShader(vs)
    glDeleteShader(fs)

In [18]:
def transform(tx, ty, tz, sx, sy, sz, rx, ry, rz):
    #matriz de translação
    translacao = np.array([
        [1.0, 0.0, 0.0, tx], 
        [0.0, 1.0, 0.0, ty], 
        [0.0, 0.0, 1.0, tz], 
        [0.0, 0.0, 0.0, 1.0]], np.float32)

    #matriz de escala
    escala = np.array([
        [sx, 0.0, 0.0, 0.0], 
        [0.0, sy, 0.0, 0.0], 
        [0.0, 0.0, sz, 0.0], 
        [0.0, 0.0, 0.0, 1.0]], np.float32)

    #matriz de rotação em torno do eixo X
    angulo = np.radians(rx)
    cos, sen = np.cos(angulo), np.sin(angulo)
    rotacaoX= np.array([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, cos, -sen, 0.0],
        [0.0, sen, cos, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])

    #matriz de rotação em torno do eixo Y
    angulo = np.radians(ry)
    cos, sen = np.cos(angulo), np.sin(angulo)
    rotacaoY= np.array([
        [cos, 0.0,sen, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [-sen, 0.0, cos, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])

    #matriz de rotação em torno do eixo Z
    angulo = np.radians(rz)
    cos, sen = np.cos(angulo), np.sin(angulo)
    rotacaoZ= np.array([
        [cos, -sen, 0.0, 0.0],
        [sen, cos, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])

    #combinacao das rotacoes
    rotacoes = rotacaoZ.dot(rotacaoY.dot(rotacaoX)) #rotação em X primeiro, seguido de Y, seguido de Z

    #matriz de transformação do cubo combinando translação, escala e as 3 rotações
    transformacao = translacao.dot(escala.dot(rotacoes)) #rotacao primeiro, escala em seguida, , translação depois

    #E passamos a matriz para o Vertex Shader.
    transformLoc = glGetUniformLocation(Shader_programm2, "matriz")
    glUniformMatrix4fv(transformLoc, 1, GL_TRUE, transformacao)

In [19]:
def especificaMatrizVisualizacao():

    # Especificação da matriz de visualização, que é definida com valores de translação e
	# rotação inversos da "posição" da câmera, pois é o mundo que se movimenta ao redor da
	# câmera, e não a câmera que se movimenta ao redor do mundo.
    visualizacao = np.identity(4)

    #posicao da camera
    translacaoCamera = np.array([
        [1.0, 0.0, 0.0, -Cam_pos[0]], 
        [0.0, 1.0, 0.0, -Cam_pos[1]], 
        [0.0, 0.0, 1.0, -Cam_pos[2]], 
        [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    #orientacao da camera (rotação em y)
    angulox = np.radians(-Cam_yaw)
    cosx, senx = np.cos(angulox), np.sin(angulox)
    rotacaoxCamera = np.array([
        [cosx, 0.0, senx, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [-senx, 0.0, cosx, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])
    
    anguloy = np.radians(Cam_pitch)
    cosy, seny = np.cos(anguloy), np.sin(anguloy)
    rotacaoyCamera = np.array([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, cosy, -seny, 0.0],
        [0.0, seny, cosy, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])
    
    visualizacao = rotacaoxCamera.dot(rotacaoyCamera.dot(translacaoCamera))

    transformLoc = glGetUniformLocation(Shader_programm2, "view")
    glUniformMatrix4fv(transformLoc, 1, GL_TRUE, visualizacao)

In [20]:
def especificaMatrizProjecao():
    #Especificação da matriz de projeção perspectiva.
    znear = 0.1 #recorte z-near
    zfar = 100.0 #recorte z-far
    fov = np.radians(67.0) #campo de visão
    aspecto = WIDTH/HEIGHT #aspecto

    a = 1/(np.tan(fov/2)*aspecto)
    b = 1/(np.tan(fov/2))
    c = (zfar + znear) / (znear - zfar)
    d = (2*znear*zfar) / (znear - zfar)
    projecao = np.array([
        [a,   0.0, 0.0,  0.0],
        [0.0, b,   0.0,  0.0],
        [0.0, 0.0, c,    d],
        [0.0, 0.0, -1.0, 1.0]
    ])

    transformLoc = glGetUniformLocation(Shader_programm2, "proj")
    glUniformMatrix4fv(transformLoc, 1, GL_TRUE, projecao)

In [21]:
def inicializaCamera():
    especificaMatrizVisualizacao()
    especificaMatrizProjecao()

In [22]:
def trataTeclado():
    global Cam_pos, Cam_yaw, Cam_pitch, Cam_yaw_speed, Cam_pitch_speed, Tempo_entre_frames
    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_ESCAPE)):
            glfw.set_window_should_close(Window, True)

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_A)):
        Cam_pos[0] -= Cam_speed * Tempo_entre_frames

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_D)):
        Cam_pos[0] += Cam_speed * Tempo_entre_frames

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_SPACE)):
        Cam_pos[1] += Cam_speed * Tempo_entre_frames

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_LEFT_SHIFT)):
        Cam_pos[1] -= Cam_speed * Tempo_entre_frames

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_W)):
        Cam_pos[2] -= Cam_speed * Tempo_entre_frames

    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_S)):
        Cam_pos[2] += Cam_speed * Tempo_entre_frames
        
    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_1)):
        toggle(0)
        
    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_2)):
        toggle(1)
        
    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_3)):
        toggle(2)
        
    if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_4)):
        toggle(3)

    xpos, ypos = glfw.get_cursor_pos(Window)
    Cam_yaw = -xpos * Cam_yaw_speed
    Cam_pitch = ((ypos - 220) * Cam_pitch_speed)
    

In [23]:
def material(KaR, KaG, KaB, KdR, KdG, KdB, KsR, KsG, KsB, n):
    global Shader_programm2
    #Coeficiente de reflexão ambiente
    Ka = np.array([KaR, KaG, KaB])#reflete 100% da luz ambiente
    Ka_loc = glGetUniformLocation(Shader_programm2, "Ka")
    glUniform3fv(Ka_loc, 1, Ka)

    #Coeficiente de reflexão difusa
    Kd = np.array([KdR, KdG, KdB])#reflete luz difusa laranja
    Kd_loc = glGetUniformLocation(Shader_programm2, "Kd")
    glUniform3fv(Kd_loc, 1, Kd)

    #Coeficiente de reflexão especular
    Ks = np.array([KsR, KsG, KsB])#reflete 100% da luz especular
    Ks_loc = glGetUniformLocation(Shader_programm2, "Ks")
    glUniform3fv(Ks_loc, 1, Ks)

    #expoente expecular
    especular_exp = n
    especular_exp_loc = glGetUniformLocation(Shader_programm2, "especular_exp")
    glUniform1f(especular_exp_loc, especular_exp)

In [24]:
def toggle(val):
    global Shader_programm2
    tog_loc = glGetUniformLocation(Shader_programm2, "toggle")
    glUniform1i(tog_loc, val)

In [25]:
def light():
    global Shader_programm
    #posição da luz
    luz_posicao = np.array([4.0, 10, 4.0])
    luz_posicaoloc = glGetUniformLocation(Shader_programm, "luz_posicao")#envia o array da posição da luz para o shader
    glUniform3fv(luz_posicaoloc, 1, luz_posicao)

    #Fonte de luz ambiente
    La = np.array([0.3, 0.3, 0.3])
    La_loc = glGetUniformLocation(Shader_programm, "La")#envia o array da Luz Ambiente para o shader
    glUniform3fv(La_loc, 1, La)

    #Fonte de luz difusa
    Ld = np.array([0.75, 0.75, 0.75])
    Ld_loc = glGetUniformLocation(Shader_programm, "Ld")#envia o array da Luz Difusa para o shader
    glUniform3fv(Ld_loc, 1, Ld)
    
    #Fonte de luz especular
    Ls = np.array([1.0, 1.0, 1.0])
    Ls_loc = glGetUniformLocation(Shader_programm, "Ls")#envia o array da Luz Especular para o shader
    glUniform3fv(Ls_loc, 1, Ls)

In [26]:
def light2():
    global Shader_programm2
    #posição da luz
    luz_posicao = np.array([4.0, 10, 4.0])
    luz_posicaoloc = glGetUniformLocation(Shader_programm2, "luz_posicao")#envia o array da posição da luz para o shader
    glUniform3fv(luz_posicaoloc, 1, luz_posicao)

    #Fonte de luz ambiente
    La = np.array([0.5, 0.15, 0.15])
    La_loc = glGetUniformLocation(Shader_programm2, "La")#envia o array da Luz Ambiente para o shader
    glUniform3fv(La_loc, 1, La)

    #Fonte de luz difusa
    Ld = np.array([0.70, 0.30, 0.30])
    Ld_loc = glGetUniformLocation(Shader_programm2, "Ld")#envia o array da Luz Difusa para o shader
    glUniform3fv(Ld_loc, 1, Ld)
    
    #Fonte de luz especular
    Ls = np.array([1.0, 1.0, 1.0])
    Ls_loc = glGetUniformLocation(Shader_programm2, "Ls")#envia o array da Luz Especular para o shader
    glUniform3fv(Ls_loc, 1, Ls)
    
    
    #Luz 2
    #-----------------------------------------------------------------------------
    #posição da luz
    luz_posicao2 = np.array([-4.0, 10, -4.0])
    luz_posicaoloc2 = glGetUniformLocation(Shader_programm2, "luz_posicao2")#envia o array da posição da luz para o shader
    glUniform3fv(luz_posicaoloc2, 1, luz_posicao2)

    #Fonte de luz ambiente
    La2 = np.array([0.15, 0.5, 0.15])
    La_loc2 = glGetUniformLocation(Shader_programm2, "La2")#envia o array da Luz Ambiente para o shader
    glUniform3fv(La_loc2, 1, La2)

    #Fonte de luz difusa
    Ld2 = np.array([0.30, 0.70, 0.30])
    Ld_loc2 = glGetUniformLocation(Shader_programm2, "Ld2")#envia o array da Luz Difusa para o shader
    glUniform3fv(Ld_loc2, 1, Ld2)
    
    #Fonte de luz especular
    Ls2 = np.array([1.0, 1.0, 1.0])
    Ls_loc2 = glGetUniformLocation(Shader_programm2, "Ls2")#envia o array da Luz Especular para o shader
    glUniform3fv(Ls_loc2, 1, Ls2)
    
    #Luz 3
    #-----------------------------------------------------------------------------
    #posição da luz
    luz_posicao3 = np.array([4.0, 10, -4.0])
    luz_posicaoloc3 = glGetUniformLocation(Shader_programm2, "luz_posicao3")#envia o array da posição da luz para o shader
    glUniform3fv(luz_posicaoloc3, 1, luz_posicao3)

    #Fonte de luz ambiente
    La3 = np.array([0.15, 0.15, 0.5])
    La_loc3 = glGetUniformLocation(Shader_programm2, "La3")#envia o array da Luz Ambiente para o shader
    glUniform3fv(La_loc3, 1, La3)

    #Fonte de luz difusa
    Ld3 = np.array([0.30, 0.30, 0.70])
    Ld_loc3 = glGetUniformLocation(Shader_programm2, "Ld3")#envia o array da Luz Difusa para o shader
    glUniform3fv(Ld_loc3, 1, Ld3)
    
    #Fonte de luz especular
    Ls3 = np.array([1.0, 1.0, 1.0])
    Ls_loc3 = glGetUniformLocation(Shader_programm2, "Ls3")#envia o array da Luz Especular para o shader
    glUniform3fv(Ls_loc3, 1, Ls3)

In [27]:
def inicializaRenderizacao():
    global Window, Shader_programm, Shader_programm2, WIDTH, HEIGHT, Tempo_entre_frames, Vao, VaoConfig, VaoMug, VaoMugConfig
    global VaoPiso, VaoPisoConfig, VaoSofa, VaoSofaConfig, VaoWall, VaoWallConfig, VaoLampada, VaoLampadaConfig
    
    tempo_anterior = glfw.get_time()

    #Ativação do teste de profundidade. Sem ele, o OpenGL não sabe que faces devem ficar na frente e que faces devem ficar atrás.
    glEnable(GL_DEPTH_TEST)
    while not glfw.window_should_close(Window):
        #calcula quantos segundos se passaram entre um frame e outro
        tempo_frame_atual = glfw.get_time()
        Tempo_entre_frames = tempo_frame_atual - tempo_anterior
        tempo_anterior = tempo_frame_atual

        #limpa a tela e os buffers
        glClearColor(0.2, 0.3, 0.3, 1.0)        
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        
        #configura a viewport para pegar toda a janela
        glViewport(0, 0, WIDTH, HEIGHT)

        #ativa o shader
        glUseProgram(Shader_programm2)
        light2() #parâmetros da fonte de luz
                                
        inicializaCamera()#configuramos a câmera
        
        
        #mesa
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(Vao) #ativamos o objeto que queremos renderizar
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 219/255, 118/255, 64/255, 0.0, 0.0, 0.0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0,0,0, 1,1,1, 0,0,0) #modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        
        #caneca
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoMug)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 232/255, 66/255, 56/255, 1.0, 1.0, 1.0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0,4,0, 1,1,1, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoMugConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #piso
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoPiso)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 149/255, 58/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0,0,0, 2.5,1,2.5, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoPisoConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        
        #teto
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoPiso)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.4, 0.4, 0.4, 232/255, 166/255, 56/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0,12,0, 2.5,1,2.5, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoPisoConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #sofa
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoSofa)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 96/255, 58/255, 0.5, 0.5, 0.5, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0,0,9, 1.4,1,1.4, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoSofaConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #sofa 2
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoSofa)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 232/255, 66/255, 56/255, 0.5, 0.5, 0.5, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(9,0,-5, 1.4,1,1.4, 0,90,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoSofaConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #parede 1
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoWall)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 232/255, 166/255, 56/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0.5,6.5,16.6, 2.5,1,2.5, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoWallConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #parede 2
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoWall)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 232/255, 166/255, 56/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(0.5,6.5,-16.6, 2.5,1,2.5, 0,0,0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoWallConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #parede 3
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoWall)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 149/255, 58/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(16.6, 6.5,  0, 2.5, 1, 2.5,  0, 90, -2.5)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoWallConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #parede 4
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoWall)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 232/255, 166/255, 56/255, 0, 0, 0, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(-16.6, 6.5,  0, 2.5, 1, 2.5,  0, 90, -2.5)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoWallConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #Luz 1
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoLampada)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 242/255, 242/255, 0.9, 0.9, 0.9, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(4, 11.5, 4,  2.5, 1, 2.5,  0, 0, 0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoLampadaConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #Luz 2
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoLampada)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 242/255, 242/255, 0.9, 0.9, 0.9, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(-4, 11.5, -4,  2.5, 1, 2.5,  0, 0, 0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoLampadaConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------
        
        #Luz 3
        #----------------------------------------------------------------------------------------------
        glBindVertexArray(VaoLampada)
        
        #cor ambiente  #cor difusa   #cor especular #n 
        material(0.2, 0.2, 0.2, 242/255, 242/255, 242/255, 0.9, 0.9, 0.9, 32) #parâmetros do material do objeto
        
        #translacao, escala, rotacao
        transform(4, 11.5, -4,  2.5, 1, 2.5,  0, 0, 0)#modifica os parâmetros de transformações geométricas para o cubo
        
        glDrawArrays(GL_TRIANGLES, 0, VaoLampadaConfig.num) #renderiza o cubo
        #----------------------------------------------------------------------------------------------

        glfw.poll_events()

        glfw.swap_buffers(Window)
        
        trataTeclado()
    
    glfw.terminate()

In [28]:
# Função principal
def main():
    inicializaOpenGL()
    inicializaObjetos()
    inicializaShaders2()
    inicializaRenderizacao()

In [29]:
main()

Placa de vídeo:  b'Intel(R) HD Graphics 630'
Versão do OpenGL:  b'4.6.0 - Build 26.20.100.8142'
